In [ ]:
from src.params import parse_arguments
from src.utils import get_device, set_seed

args = parse_arguments()
args.device = get_device(force_cpu=True)
print(args)
set_seed(args.seed)

/home/x_yxiao/.conda/envs/ai-pet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting project path based on the current script directory
Project path set to: /proj/berzelius-2024-156/users/x_yxiao/AI-PET
Program starts at 01-10-2025 23:45:36
Output directory created at: /proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_visual_read_hypertune-optuna-60trials_stratify-visual_read,site_IDEAS_Inten_Norm_20251001_234536
Namespace(model='CNN3D', model_name_extra='IDEAS_Inten_Norm', input_path='/proj/berzelius-2024-156/users/x_yxiao/AI-PET/data', data_suffix='', targets='visual_read', model_kwargs='', lr=0.0001, weight_decay=0.0001, batch_size=4, dropout=0.3, epochs=200, loss_w_cls=1.0, loss_w_reg=1.0, num_workers=8, resume='', amp=False, seed=42, n_splits=5, stratifycvby='visual_read,site', image_shape=[128, 128, 128], tune=True, n_trials=60, proxy_epochs=60, proxy_folds=5, study_name='optuna', storage='', tune_timeout=None, output_date_time='20251001_234536', proj_path='/proj/berzelius-2024-156/users/x_yxiao/AI-PET', script_path='/proj/berzelius-2024-156/use

In [ ]:
from src.utils import build_model_from_args
model = build_model_from_args(args, device=args.device, n_classes=2)

import torch
sd = torch.load('/proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_visual_read_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251001_082228/outer-test/checkpoints/best.pt', map_location=args.device, weights_only=True)
state_dict = sd.get("model", sd) if isinstance(sd, dict) else sd
model.load_state_dict(state_dict, strict=False)

in_channels: 1, widths: (32, 64, 128, 256), pool_every: 1, dropout: 0.3, norm: batch, num_classes: 2
CNN3D(
  (features): Sequential(
    (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (8): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (11): BatchNorm3d(64, eps=1e-05, momentum=0

<All keys matched successfully>

In [ ]:
import pandas as pd
val_df = pd.read_csv('/proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_visual_read_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251001_082228/Hold-out_testing-set.csv', index_col=0)
display(val_df)
from src.data import PETDataset
from torch.utils.data import DataLoader
from src.utils import seed_worker
def get_loader(df, tfm, args, batch_size, augment=False, shuffle=False):
    g = torch.Generator()
    g.manual_seed(args.seed)

    dataset = PETDataset(df, tfm, args.targets, augment=augment)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                       worker_init_fn=seed_worker, generator=g,
                       num_workers=args.num_workers, pin_memory=False)

    return loader

tfm =tfm = args.input_path
dl_va = get_loader(val_df, tfm, args, batch_size=max(1, args.batch_size // 2), augment=False, shuffle=False)

,ID,site,tracer,CL,visual_read,visual_read_confidence,age,gender,ethnicity,education,diagnosis,AD_pathology,APOE
0,3,17601,1.0,35.0,0.0,3.0,73,2,1,3,13.0,8.0,0
1,7,20004,1.0,14.4,1.0,3.0,74,1,1,6,23.0,4.0,0
2,15,20354,2.0,75.8,0.0,3.0,68,1,4,7,11.0,7.0,0
3,24,20423,2.0,92.6,0.0,3.0,83,1,1,8,11.0,7.0,0
4,35,17601,2.0,1.8,1.0,2.0,74,2,1,8,11.0,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,10329,21172,2.0,93.0,0.0,3.0,68,2,1,4,11.0,7.0,0
2065,10330,17515,1.0,-17.0,1.0,2.0,72,2,1,4,13.0,3.0,0
2066,10332,17601,2.0,27.4,0.0,3.0,81,1,1,5,14.0,7.0,0
2067,10333,17601,2.0,113.2,0.0,3.0,73,1,1,5,11.0,4.0,0


In [4]:
from src.cam import find_last_conv3d, GradCAM3D

# Grad-CAM snapshots for the best model on validation set
target_layer = find_last_conv3d(model)  # generic for CNN3D, UNet3D, etc.
        
task = "cls"
cam = GradCAM3D(model, target_layer=target_layer, task=task)


In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def save_gradcam_overlays_avg_by_slice(
    model, cam, loader, out_dir, device, max_items=None,
    slice_fracs=(0.25, 0.5, 0.75), overlay_cam=True,
    clip_percentile=0.5, cmap_img="gray", cam_alpha=0.4
):
    """
    Computes the average volume (and average CAM) across subjects, then
    saves ONE figure per slice position. Each figure has 3 subplots:
    Axial / Coronal / Sagittal for that slice position.

    Args
    ----
    model, cam, loader, device : as usual
    out_dir : str
    max_items : int | None  cap subjects used (None = all)
    slice_fracs : tuple[float]  positions in [0,1] along each axis
    overlay_cam : bool  overlay mean CAM if True
    clip_percentile : float  robust intensity scaling (0–2 recommended)
    cmap_img : str  matplotlib cmap for base image
    cam_alpha : float  overlay transparency
    """
    os.makedirs(out_dir, exist_ok=True)

    sum_vol, sum_cam, n_seen = None, None, 0

    for x, y_cls, y_reg, pid in loader:
        if max_items is not None and n_seen >= max_items:
            break

        b = x.shape[0]
        if max_items is not None:
            b = min(b, max_items - n_seen)
            x = x[:b]

        x = x.to(device, non_blocking=True)

        with torch.enable_grad():
            heat = cam(x)  # [B,1,D,H,W] (will resize)

        if heat.shape[2:] != x.shape[2:]:
            heat = F.interpolate(heat, size=x.shape[2:], mode="trilinear", align_corners=False)

        x_np = x.detach().cpu().numpy()[:, 0]      # [B, D, H, W]
        h_np = heat.detach().cpu().numpy()[:, 0]   # [B, D, H, W]

        if sum_vol is None:
            sum_vol = np.zeros_like(x_np[0], dtype=np.float64)
            if overlay_cam:
                sum_cam = np.zeros_like(h_np[0], dtype=np.float64)

        sum_vol += x_np[:b].sum(axis=0)  # true sum
        if overlay_cam:
            sum_cam += h_np[:b].sum(axis=0)

        n_seen += b
    print('finish loading')
    if n_seen == 0:
        print("No data encountered — nothing saved.")
        return

    mean_vol = sum_vol / n_seen
    mean_cam = (sum_cam / n_seen) if overlay_cam else None

    D, H, W = mean_vol.shape

    def frac_to_idx(f, L):
        f = float(np.clip(f, 0.0, 1.0))
        return int(round(f * (L - 1)))

    # robust display range for the mean image
    if clip_percentile is not None:
        lo = np.percentile(mean_vol, max(0, 50 - clip_percentile * 50))
        hi = np.percentile(mean_vol, min(100, 50 + clip_percentile * 50))
        lo, hi = float(lo), float(hi)
    else:
        lo, hi = float(mean_vol.min()), float(mean_vol.max())

    # optional CAM normalization per-view for nicer overlays
    def norm_cam(arr):
        vmin, vmax = np.percentile(arr, 1), np.percentile(arr, 99)
        vmax = vmax if vmax > vmin else vmin + 1e-6
        return np.clip((arr - vmin) / (vmax - vmin + 1e-12), 0, 1)

    # ---- Save one figure per slice fraction ----
    for f in slice_fracs:
        z = frac_to_idx(f, D)
        y = frac_to_idx(f, H)
        xidx = frac_to_idx(f, W)

        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        # Axial
        axes[0].imshow(mean_vol[z, :, :], cmap=cmap_img, vmin=lo, vmax=hi)
        if overlay_cam:
            axes[0].imshow(norm_cam(mean_cam[z, :, :]), alpha=cam_alpha)
        axes[0].set_title(f"Axial (z={z}, f={f:.2f})")
        axes[0].axis("off")

        # Coronal
        axes[1].imshow(mean_vol[:, y, :], cmap=cmap_img, vmin=lo, vmax=hi)
        if overlay_cam:
            axes[1].imshow(norm_cam(mean_cam[:, y, :]), alpha=cam_alpha)
        axes[1].set_title(f"Coronal (y={y}, f={f:.2f})")
        axes[1].axis("off")

        # Sagittal
        axes[2].imshow(mean_vol[:, :, xidx], cmap=cmap_img, vmin=lo, vmax=hi)
        if overlay_cam:
            axes[2].imshow(norm_cam(mean_cam[:, :, xidx]), alpha=cam_alpha)
        axes[2].set_title(f"Sagittal (x={xidx}, f={f:.2f})")
        axes[2].axis("off")

        plt.tight_layout()
        fname = os.path.join(out_dir, f"avg_axcorSag_f{f:.2f}.png")
        plt.savefig(fname, dpi=150)
        plt.close(fig)

    print(f"Saved {len(slice_fracs)} figures (n={n_seen}) to: {out_dir}")


In [ ]:
save_gradcam_overlays_avg_by_slice(model, cam, dl_va, '/proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_visual_read_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251001_082228/test_vis_new/', args.device)